# 2. OpenAI の チャット API の基礎


## 2.3. 入出力の長さの制限や料金に影響する「トークン」


### トークン


In [ ]:
!pip install tiktoken==0.7.0

In [ ]:
import tiktoken

text = "ChatGPT"

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
for token in tokens:
    print(encoding.decode([token]))

### Tokenizer と tiktoken の紹介


In [ ]:
import tiktoken

text = "LLMを使ってクールなものを作るのは簡単だが、プロダクションで使えるものを作るのは非常に難しい。"

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
print(len(tokens))

## 2.4. Chat Completions API を試す環境の準備


### OpenAI の API キーの準備


In [8]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## 2.5. Chat Completions API のハンズオン


### OpenAI のライブラリ


#### 【注意】既知のエラーについて

openai パッケージが依存する httpx のアップデートにより、`openai==1.40.6` を使用する箇所で `TypeError: Client.__init__() got an unexpected keyword argument 'proxies'` というエラーが発生するようになりました。

このエラーは、`!pip install httpx==0.27.2` のように、httpx の特定バージョンをインストールすることで回避することができます。

なお、Google Colab で一度上記のエラーに遭遇したあとで `!pip install httpx==0.27.2` のようにパッケージをインストールし直した場合、以下のどちらかの操作を実施する必要があります。

- Google Colab の「ランタイム」から「セッションを再起動する」を実行する
- 「ランタイムを接続解除して削除」を実行してパッケージのインストールからやり直す


In [ ]:
!pip install openai==1.40.6 httpx==0.27.2

### Chat Completions API の呼び出し


In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは！私はジョンと言います！"},
    ],
)
print(response.to_json(indent=2))

### 会話履歴を踏まえた応答を得る


In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは！私はジョンと言います！"},
        {"role": "assistant", "content": "こんにちは、ジョンさん！お会いできて嬉しいです。今日はどんなことをお話ししましょうか？"},
        {"role": "user", "content": "私の名前が分かりますか？"},
    ],
)
print(response.to_json(indent=2))

### ストリーミングで応答を得る


In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは！私はジョンと言います！"},
    ],
    stream=True,
)

for chunk in response:
    content = chunk.choices[0].delta.content
    if content is not None:
        print(content, end="", flush=True)

### JSON モード


In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": '人物一覧を次のJSON形式で出力してください。\n{"people": ["aaa", "bbb"]}',
        },
        {
            "role": "user",
            "content": "昔々あるところにおじいさんとおばあさんがいました",
        },
    ],
    response_format={"type": "json_object"},
)
print(response.choices[0].message.content)

### Vision（画像入力）


In [ ]:
from openai import OpenAI

client = OpenAI()

image_url = "https://raw.githubusercontent.com/yoshidashingo/langchain-book/main/assets/cover.jpg"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "画像を説明してください。"},
                {"type": "image_url", "image_url": {"url": image_url}},
            ],
        }
    ],
)

print(response.choices[0].message.content)

### （コラム）Completions API


In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt="こんにちは！私はジョンと言います！",
)
print(response.to_json(indent=2))

## 2.6. Function calling


### Function calling のサンプルコード


In [ ]:
import json


def get_current_weather(location, unit="fahrenheit"):
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": "72", "unit": unit}
        )
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [ ]:
from openai import OpenAI

client = OpenAI()

messages = [
    {"role": "user", "content": "東京の天気はどうですか？"},
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)
print(response.to_json(indent=2))

In [ ]:
response_message = response.choices[0].message
messages.append(response_message.to_dict())

In [ ]:
available_functions = {
    "get_current_weather": get_current_weather,
}

# 使いたい関数は複数あるかもしれないのでループ
for tool_call in response_message.tool_calls:
    # 関数を実行
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )
    print(function_response)

    # 関数の実行結果を会話履歴としてmessagesに追加
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

In [ ]:
print(json.dumps(messages, ensure_ascii=False, indent=2))

In [ ]:
second_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
)
print(second_response.to_json(indent=2))